In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('Data/train.csv')
test_data = pd.read_csv('Data/test.csv')

In [50]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [35]:
text_data = train_data['text'].to_list()
labels_data = train_data['target'].to_list()

train_tweets = text_data[:int(len(text_data)*0.8)]
train_labels = labels_data[:int(len(labels_data)*0.8)]

val_tweets = text_data[int(len(text_data)*0.8):]
val_labels = labels_data[int(len(labels_data)*0.8):]

test_tweets = test_data['text'].to_list()

In [6]:
vocab_size = 10000
embedding_dim = 16
max_length = 80
trunc_type = 'post'
oov_token = '<OOV>'

In [40]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_tweets)

sequences = tokenizer.texts_to_sequences(train_tweets)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

val_sequences = tokenizer.texts_to_sequences(val_tweets)
val_padded = pad_sequences(val_sequences, maxlen=max_length, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_tweets)
padded_test = pad_sequences(test_sequences, maxlen=max_length)

In [41]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [42]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [43]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 80, 16)            160000    
_________________________________________________________________
flatten_5 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 6)                 7686      
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 7         
Total params: 167,693
Trainable params: 167,693
Non-trainable params: 0
_________________________________________________________________


In [45]:
num_epochs = 10
model.fit(padded, train_labels, epochs=num_epochs, validation_data=(val_padded, val_labels))

Train on 6090 samples, validate on 1523 samples
Epoch 1/10
6090/6090 [==============================] - 0s 80us/sample - loss: 0.6491 - accuracy: 0.6107 - val_loss: 0.6311 - val_accuracy: 0.6441
Epoch 2/10
6090/6090 [==============================] - 0s 51us/sample - loss: 0.5159 - accuracy: 0.7836 - val_loss: 0.5011 - val_accuracy: 0.7689
Epoch 3/10
6090/6090 [==============================] - 0s 52us/sample - loss: 0.2920 - accuracy: 0.8946 - val_loss: 0.5176 - val_accuracy: 0.7663
Epoch 4/10
6090/6090 [==============================] - 0s 47us/sample - loss: 0.1749 - accuracy: 0.9440 - val_loss: 0.4813 - val_accuracy: 0.7859
Epoch 5/10
6090/6090 [==============================] - 0s 47us/sample - loss: 0.1121 - accuracy: 0.9670 - val_loss: 0.4999 - val_accuracy: 0.7846
Epoch 6/10
6090/6090 [==============================] - 0s 47us/sample - loss: 0.0809 - accuracy: 0.9759 - val_loss: 0.5176 - val_accuracy: 0.7722
Epoch 7/10
6090/6090 [==============================] - 0s 48us/sample

In [60]:
predicted_classes = model.predict_classes(padded_test)

In [65]:
first_result = pd.DataFrame()
first_result['id'] = test_data['id']
first_result['target'] = predicted_classes

In [69]:
first_result.to_csv('first_results.csv', index=False)

Model 2

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
#     tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [77]:
num_epochs = 10
model.fit(padded, train_labels, epochs=num_epochs)

Train on 7613 samples
Epoch 1/10
7613/7613 [==============================] - 0s 58us/sample - loss: 0.6753 - accuracy: 0.5707
Epoch 2/10
7613/7613 [==============================] - 0s 45us/sample - loss: 0.5876 - accuracy: 0.7112
Epoch 3/10
7613/7613 [==============================] - 0s 45us/sample - loss: 0.4586 - accuracy: 0.8049
Epoch 4/10
7613/7613 [==============================] - 0s 45us/sample - loss: 0.3801 - accuracy: 0.8479
Epoch 5/10
7613/7613 [==============================] - 0s 45us/sample - loss: 0.3320 - accuracy: 0.8663
Epoch 6/10
7613/7613 [==============================] - 0s 44us/sample - loss: 0.2942 - accuracy: 0.8840
Epoch 7/10
7613/7613 [==============================] - 0s 44us/sample - loss: 0.2655 - accuracy: 0.8961
Epoch 8/10
7613/7613 [==============================] - 0s 44us/sample - loss: 0.2388 - accuracy: 0.9082
Epoch 9/10
7613/7613 [==============================] - 0s 43us/sample - loss: 0.2165 - accuracy: 0.9172
Epoch 10/10
7613/7613 [==========

### Now Trying with LSTM layer

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
#     tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
num_epochs = 10
model.fit(padded, train_labels, epochs=num_epochs)

Train on 7613 samples
Epoch 1/10
7613/7613 [==============================] - 7s 889us/sample - loss: 0.5895 - accuracy: 0.6816
Epoch 2/10
7613/7613 [==============================] - 6s 832us/sample - loss: 0.3646 - accuracy: 0.8482
Epoch 3/10
7613/7613 [==============================] - 6s 815us/sample - loss: 0.2678 - accuracy: 0.8968
Epoch 4/10
7613/7613 [==============================] - 6s 820us/sample - loss: 0.2080 - accuracy: 0.9257
Epoch 5/10
7613/7613 [==============================] - 6s 816us/sample - loss: 0.1669 - accuracy: 0.9433
Epoch 6/10
7613/7613 [==============================] - 6s 819us/sample - loss: 0.1350 - accuracy: 0.9543
Epoch 7/10
7613/7613 [==============================] - 6s 824us/sample - loss: 0.1160 - accuracy: 0.9602
Epoch 8/10
7613/7613 [==============================] - 6s 842us/sample - loss: 0.1028 - accuracy: 0.9648
Epoch 9/10
7613/7613 [==============================] - 6s 828us/sample - loss: 0.0870 - accuracy: 0.9703
Epoch 10/10
7613/7613 [=

With flatten

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim), return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
num_epochs = 10
model.fit(padded, train_labels, epochs=num_epochs)

Train on 7613 samples
Epoch 1/10
7613/7613 [==============================] - 7s 929us/sample - loss: 0.5665 - accuracy: 0.7006
Epoch 2/10
7613/7613 [==============================] - 7s 864us/sample - loss: 0.3544 - accuracy: 0.8555
Epoch 3/10
7613/7613 [==============================] - 7s 873us/sample - loss: 0.2630 - accuracy: 0.9006
Epoch 4/10
7613/7613 [==============================] - 7s 884us/sample - loss: 0.2078 - accuracy: 0.9215
Epoch 5/10
7613/7613 [==============================] - 7s 867us/sample - loss: 0.1631 - accuracy: 0.9421
Epoch 6/10
7613/7613 [==============================] - 7s 856us/sample - loss: 0.1322 - accuracy: 0.9542
Epoch 7/10
7613/7613 [==============================] - 7s 958us/sample - loss: 0.1092 - accuracy: 0.9640
Epoch 8/10
7613/7613 [==============================] - 7s 859us/sample - loss: 0.0935 - accuracy: 0.9703
Epoch 9/10
7613/7613 [==============================] - 7s 863us/sample - loss: 0.0856 - accuracy: 0.9710
Epoch 10/10
7613/7613 [=

With 2 LSTM layers

In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(int(embedding_dim/2))),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
num_epochs = 10
model.fit(padded, train_labels, epochs=num_epochs)

Train on 7613 samples
Epoch 1/10
7613/7613 [==============================] - 12s 2ms/sample - loss: 0.6157 - accuracy: 0.6549
Epoch 2/10
7613/7613 [==============================] - 11s 1ms/sample - loss: 0.3836 - accuracy: 0.8432
Epoch 3/10
7613/7613 [==============================] - 10s 1ms/sample - loss: 0.2870 - accuracy: 0.8914
Epoch 4/10
7613/7613 [==============================] - 11s 1ms/sample - loss: 0.2217 - accuracy: 0.9233
Epoch 5/10
7613/7613 [==============================] - 11s 1ms/sample - loss: 0.1768 - accuracy: 0.9392
Epoch 6/10
7613/7613 [==============================] - 11s 1ms/sample - loss: 0.1406 - accuracy: 0.9532
Epoch 7/10
7613/7613 [==============================] - 11s 1ms/sample - loss: 0.1116 - accuracy: 0.9622
Epoch 8/10
7613/7613 [==============================] - 11s 1ms/sample - loss: 0.0906 - accuracy: 0.9648
Epoch 9/10
7613/7613 [==============================] - 12s 2ms/sample - loss: 0.0745 - accuracy: 0.9694
Epoch 10/10
7613/7613 [==========

With Dropout

In [47]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(int(embedding_dim/2))),
#     tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
num_epochs = 10
model.fit(padded, train_labels, epochs=num_epochs, validation_data=(val_padded, val_labels))

Train on 7613 samples
Epoch 1/10
7613/7613 [==============================] - 13s 2ms/sample - loss: 0.5831 - accuracy: 0.6929
Epoch 2/10
7613/7613 [==============================] - 11s 1ms/sample - loss: 0.3723 - accuracy: 0.8530
Epoch 3/10
7613/7613 [==============================] - 11s 1ms/sample - loss: 0.2818 - accuracy: 0.9011
Epoch 4/10
7613/7613 [==============================] - 12s 2ms/sample - loss: 0.2265 - accuracy: 0.9226
Epoch 5/10
7613/7613 [==============================] - 13s 2ms/sample - loss: 0.1841 - accuracy: 0.9426
Epoch 6/10
7613/7613 [==============================] - 11s 1ms/sample - loss: 0.1586 - accuracy: 0.9507
Epoch 7/10
7613/7613 [==============================] - 11s 1ms/sample - loss: 0.1378 - accuracy: 0.9561
Epoch 8/10
7613/7613 [==============================] - 11s 1ms/sample - loss: 0.1201 - accuracy: 0.9640
Epoch 9/10
7613/7613 [==============================] - 11s 1ms/sample - loss: 0.1019 - accuracy: 0.9685
Epoch 10/10
7613/7613 [==========

In [48]:
num_epochs = 10
model.fit(padded, train_labels, epochs=num_epochs, validation_data=(val_padded, val_labels))

Train on 6090 samples, validate on 1523 samples
Epoch 1/10
6090/6090 [==============================] - 12s 2ms/sample - loss: 0.5925 - accuracy: 0.6882 - val_loss: 0.4701 - val_accuracy: 0.7873
Epoch 2/10
6090/6090 [==============================] - 10s 2ms/sample - loss: 0.3734 - accuracy: 0.8517 - val_loss: 0.4531 - val_accuracy: 0.7938
Epoch 3/10
6090/6090 [==============================] - 10s 2ms/sample - loss: 0.2709 - accuracy: 0.9036 - val_loss: 0.5314 - val_accuracy: 0.7820
Epoch 4/10
6090/6090 [==============================] - 10s 2ms/sample - loss: 0.2058 - accuracy: 0.9307 - val_loss: 0.5784 - val_accuracy: 0.7774
Epoch 5/10
6090/6090 [==============================] - 10s 2ms/sample - loss: 0.1589 - accuracy: 0.9507 - val_loss: 0.6028 - val_accuracy: 0.7695
Epoch 6/10
6090/6090 [==============================] - 10s 2ms/sample - loss: 0.1355 - accuracy: 0.9589 - val_loss: 0.6755 - val_accuracy: 0.7564
Epoch 7/10
6090/6090 [==============================] - 10s 2ms/sample

The results are getting worse. So the first way to make the NN is the best for now.  
Next Step:  
- Clean data
- Integrate the other columns to the data